# 1) How to display a data frame in PyQt5 Window 

In [1]:
#https://stackoverflow.com/questions/44603119/how-to-display-a-pandas-data-frame-with-pyqt5
import pandas as pd
import re
import sys
import os
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import (QMainWindow, QApplication, QPushButton, QWidget,  
                             QVBoxLayout, QHBoxLayout, QPushButton, QTextEdit, QTableView)
from PyQt5.QtGui import  QTextCursor
from PyQt5.QtCore import QCoreApplication, QAbstractTableModel, QVariant, QModelIndex

### data must be provided by creating a model to display data frame in GUI window...

In [2]:
class PandasModel(QAbstractTableModel): 
    def __init__(self, df = pd.DataFrame(), parent=None): 
        QAbstractTableModel.__init__(self, parent=parent)
        self._df = df

    def headerData(self, section, orientation, role=Qt.DisplayRole):
        if role != Qt.DisplayRole:
            return QVariant()

        if orientation == Qt.Horizontal:
            try:
                return self._df.columns.tolist()[section]
            except (IndexError, ):
                return QVariant()
        elif orientation == Qt.Vertical:
            try:
                # return self.df.index.tolist()
                return self._df.index.tolist()[section]
            except (IndexError, ):
                return QVariant()

    def data(self, index, role=Qt.DisplayRole):
        if role != Qt.DisplayRole:
            return QVariant()

        if not index.isValid():
            return QVariant()

        return QVariant(str(self._df.ix[index.row(), index.column()]))

    def setData(self, index, value, role):
        row = self._df.index[index.row()]
        col = self._df.columns[index.column()]
        if hasattr(value, 'toPyObject'):
            # PyQt4 gets a QVariant
            value = value.toPyObject()
        else:
            # PySide gets an unicode
            dtype = self._df[col].dtype
            if dtype != object:
                value = None if value == '' else dtype.type(value)
        self._df.set_value(row, col, value)
        return True

    def rowCount(self, parent=QModelIndex()): 
        return len(self._df.index)

    def columnCount(self, parent=QModelIndex()): 
        return len(self._df.columns)

    def sort(self, column, order):
        colname = self._df.columns.tolist()[column]
        self.layoutAboutToBeChanged.emit()
        self._df.sort_values(colname, ascending= order == QtCore.Qt.AscendingOrder, inplace=True)
        self._df.reset_index(inplace=True, drop=True)
        self.layoutChanged.emit()

### when you click 'create dataframe', program will look into file path & run neuro_xml_to_df & display output in GUI window

In [3]:
class App(QMainWindow):
    
    
    def __init__(self):
        super(App, self).__init__()
        self.setGeometry(50, 50, 500, 300)
        self.setWindowTitle("PyQT tuts!")
        self.home()

    def home(self):
        
        # create a simple layout with a button 
        w = QWidget()
        self.setCentralWidget(w)
        lay = QVBoxLayout(w)
        btn = QPushButton("Create Dataframe")
        btn.clicked.connect(self.df_handler)
        
        # pandas stuff 
        self.tableView = QTableView()
        self.tableView.setObjectName("tableView")
        

        self.process  = QTextEdit()
        self.process.moveCursor(QTextCursor.Start)
        self.process.ensureCursorVisible()
        self.process.setLineWrapColumnOrWidth(1000)
        self.process.setLineWrapMode(QTextEdit.FixedPixelWidth)

        lay.addWidget(btn)
        lay.addWidget(self.process)
        lay.addWidget(self.tableView)

        self.show()
        
    def df_handler(self, signal):
        
        df = self.neuro_xml_to_df('/vol01/raw_data/staging/indiana')
        model = PandasModel(df)
        self.tableView.setModel(model)
    
    # this function is a test to see if stdout/stderr gets sent to GUI window 
    def neuro_xml_to_df(self, path):
        """Given a full path to neuropsych sub folders, returns data frame of first 6 lines of xml file"""
        self.path = path

        xml = [os.path.join(root,name) for root,dirs,files in os.walk(path) for name in files if name.endswith(".xml")]

        ids_lst=[]
        dob_lst = []
        gen_lst=[]
        test_lst =[]
        ses_lst=[]
        han_lst=[]
        for i in xml:
            with open(i) as f:
                for line in f:
                    if line.startswith('  <Sub'):
                        ids_lst.extend(re.findall(r'<SubjectID>(.*?)</SubjectID>', line))
                    if line.startswith('  <DOB'):
                        dob_lst.extend(re.findall(r'<DOB>(.*?)</DOB>', line))
                    if line.startswith('  <Gen'):
                        gen_lst.extend(re.findall(r'<Gender>(.*?)</Gender>', line))
                    if line.startswith('  <Test'):
                        test_lst.extend(re.findall(r'<TestDate>(.*?)</TestDate>', line))
                    if line.startswith('  <Sess'):
                        ses_lst.extend(re.findall(r'<SessionCode>(.*?)</SessionCode>', line))
                    if line.startswith('  <Hand'):
                        han_lst.extend(re.findall(r'<Hand>(.*?)</Hand>', line))

        data_set = pd.DataFrame(ids_lst, columns=["Subject ID"])
        data_set['Test_Date'] = test_lst
        data_set['DOB'] = dob_lst
        data_set['Gender'] = gen_lst
        data_set['Handedness'] = han_lst
        data_set['Run Letter'] = ses_lst

        data_set['Test_Date'] =pd.to_datetime(data_set.Test_Date)
        table = data_set.sort_values('Test_Date', ascending=True)
        print("sorting by test date...")
        return table


if __name__ == '__main__':
    app = QCoreApplication.instance() ### adding this if statement prevents kernel from crashing 
    if app is None:
        app = QApplication(sys.argv)
        print(app)
    ex = App()
    sys.exit(app.exec_())

sorting by test date...


/usr/local/anaconda3/envs/dbI/lib/python3.5/site-packages/ipykernel/__main__.py:29: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


SystemExit: 0

/usr/local/anaconda3/envs/dbI/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 2) redirect terminal output to QtextEdit & button to clear window

In [1]:
#https://stackoverflow.com/questions/44432276/print-out-python-console-output-to-qtextedit
import sys
import os
import subprocess
from PyQt5.QtWidgets import (QMainWindow, QApplication, QPushButton, QWidget,  
                             QVBoxLayout, QHBoxLayout, QLabel, QTextEdit)
from PyQt5.QtGui import QTextCursor
from PyQt5.QtCore import QCoreApplication, QProcess, QObject, pyqtSignal

In [2]:
class Stream(QObject):
    newText = pyqtSignal(str)

    def write(self, text):
        self.newText.emit(str(text))
    
    def flush(self):
        pass
        
class App(QMainWindow):
    
    
    def __init__(self):
        super(App, self).__init__()
        self.setGeometry(50, 50, 500, 300)
        self.setWindowTitle("PyQT tuts!")
        self.home()

        sys.stdout = Stream(newText=self.onUpdateText)

    def onUpdateText(self, text):
        cursor = self.process.textCursor()
        cursor.movePosition(QTextCursor.End)
        cursor.insertText(text)
        self.process.setTextCursor(cursor)
        self.process.ensureCursorVisible()

    def __del__(self):
        sys.stdout = sys.__stdout__

    def home(self):
        
        # create a window 
        w = QWidget()
        self.setCentralWidget(w)
        
        # add a push button connected to something
        lay = QVBoxLayout(w)
        btn = QPushButton("Click me to see redirected terminal output")
        btn.clicked.connect(self.handler)
        
        # add a clear button
        clear_button = QPushButton('Clear all text')
        clear_button.clicked.connect(self.clear_function)
        
        # add text editor widget to provide a place for terminal output to be redirected to 
        self.process  = QTextEdit()
        self.process.moveCursor(QTextCursor.Start)
        self.process.ensureCursorVisible()
        self.process.setLineWrapColumnOrWidth(1000)
        self.process.setLineWrapMode(QTextEdit.FixedPixelWidth)
        
        # add widgets to GUI
        lay.addWidget(btn)
        lay.addWidget(clear_button)
        lay.addWidget(self.process)

        self.show()
 
    # this method clears Qtextedit window
    def clear_function(self, signal):
        self.process.clear()
        
        
    def handler(self, signal):
        self.erp_and_filesize_check('/vol01/active_projects/anthony/ns650')
    
        
    # this function is a test to see if stdout/stderr gets sent to GUI window 
    def erp_and_filesize_check(self, fp_check):
        self.fp_check = fp_check

        for r,d,f in os.walk(fp_check):
            for n in d:
                dirs = os.path.join(r,n)
                erp_check = subprocess.Popen('ERP-raw-data_check.sh {}'.format(dirs), shell=True, 
                                              stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=dirs)
                result_erp = erp_check.communicate()[0]
                print('ERP CHECK:', dirs, result_erp.decode('ascii'))
                size_check = subprocess.Popen('DVD-file-size_check.sh {}'.format(dirs), shell=True, 
                                              stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=dirs)
                result_size = size_check.communicate()[0]
                print('FILE SIZE CHECK: ', dirs, result_size.decode('ascii'))
        
        
if __name__ == '__main__':
    app = QCoreApplication.instance() ### adding this if statement prevents kernel from crashing 
    if app is None:
        app = QApplication(sys.argv)
        print(app)
    ex = App()
    sys.exit(app.exec_())

SystemExit: 0

/usr/local/anaconda3/envs/dbI/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
